In [ ]:
from collections import namedtuple
import pathlib
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
def collect_time_series(so_far, ts):
    return toolz.merge(so_far, {ts.DisplayName: ts})

def well_time_series(project):
    return toolz.reduce(collect_time_series, project.WellTimeSeriesList.Items, {})

project_time_series = toolz.valmap(well_time_series, projects)
project_time_series

In [ ]:
toolz.valmap(len, project_time_series)

In [ ]:
TimeSeriesSummary = namedtuple('TimeSeriesSummary', ['display_name', 'name',
                                                     'phenomenon', 'net_unit', 'suffix',
                                                     'start_time', 'stop_time'])


In [ ]:
def summarize_time_series(ts):
    time_series_range = ts.GetTimeSeriesRange()
    start = time_series_range.Item1
    stop = time_series_range.Item2
    return TimeSeriesSummary(ts.DisplayName, ts.Name,
                             ts.SampledQuantityName, ts.SampledQuantityType, ts.Suffix,
                             start, stop)

project_time_series_summaries = toolz.valmap(
    toolz.valmap(summarize_time_series),
    project_time_series)
project_time_series_summaries

In [ ]:
project_time_series_samples = toolz.valmap(
    toolz.valmap(toolz.compose(# list,
                               lambda ts: ts.GetOrderedTimeSeriesHistory())),
    project_time_series)
# samples_to_print = toolz.valmap(
#     toolz.map(len),
#     project_time_series_samples)
# samples_to_print

In [ ]:
def generate_sample_indices(ts):
    candidate_indices = list(range(len(ts)))
    random.shuffle(candidate_indices)
    inner_indices = sorted(candidate_indices[:6])
    result = [0] + inner_indices + [len(ts) - 1]
    return result

project_sample_indices = toolz.valmap(toolz.valmap(generate_sample_indices),
                                      project_time_series_samples)
project_sample_indices

In [ ]:
MergedSamples = namedtuple('MergedSamples', ['samples', 'summary', 'indices'])

def merge_monitor_series_info(samples, summaries, indices):
    return toolz.merge_with(lambda ks: MergedSamples(*ks),
                            samples, summaries, indices)

def merge_field_series_info(field_name):
    return merge_monitor_series_info(project_time_series_samples[field_name],
                                     project_time_series_summaries[field_name],
                                     project_sample_indices[field_name])

In [ ]:
bakken_time_series_merged_info = merge_field_series_info('bakken')
bakken_time_series_merged_info
# montney_time_series_merged_info = merge_field_series_info('montney')
# montney_time_series_merged_info

In [ ]:
@toolz.curry
def make_sample_pressure(field_name, sample):
    return UnitsNet.Pressure.From(UnitsNet.QuantityValue.op_Implicit(sample.Value),
                                  project_units[field_name].PressureUnit)

@toolz.curry
def sample_time_series(field_name, merged_info):
    result = toolz.pipe(merged_info.indices,
                        # toolz.map(toolz.do(lambda n: print(n))),
                        toolz.map(lambda n: toolz.nth(n, merged_info.samples)),
                        # toolz.map(toolz.do(lambda s: print(s.Value))),
                        toolz.map(make_sample_pressure(field_name)),

                        toolz.map(str),
                        list,
                        )
    return result

bakken_time_series_monitor_info = toolz.valmap(sample_time_series('bakken'),
                                       bakken_time_series_merged_info)
bakken_time_series_monitor_info

In [ ]:
@toolz.curry
def make_series_info(field_name, monitor_name, samples):
    result = {'field': field_name,
              'monitor': monitor_name,
              'samples': samples}
    return result

@toolz.curry
def make_monitor_series_info(field_name, item):
    monitor_name, samples = item
    return make_series_info(field_name, monitor_name, samples)

def field_series_info(field_name, monitor_info):
    return [make_monitor_series_info(field_name, item) for item in monitor_info.items()]

bakken_time_series_info = field_series_info('bakken', bakken_time_series_monitor_info)
bakken_time_series_info

In [ ]:
pd.DataFrame(data=bakken_time_series_info)